# <p style="background-color:lightgray; font-family:verdana; font-size:250%; text-align:center; border-radius: 15px 20px;">🟠Libraries and Data import -- First look at Data o.O 🟠</p>

In [491]:
import numpy as np 
import pandas as pd 
from surprise import Reader, SVD, Dataset, accuracy
from surprise.model_selection import GridSearchCV, train_test_split, cross_validate

In [492]:
books = pd.read_csv("/kaggle/input/goodbooks-10k/books.csv", 
                 usecols=["book_id",
                          "original_publication_year",
                          "average_rating",
                          "title",
                          "average_rating"])
books.head(5)

,book_id,original_publication_year,title,average_rating
0,2767052,2008.0,"The Hunger Games (The Hunger Games, #1)",4.34
1,3,1997.0,Harry Potter and the Sorcerer's Stone (Harry P...,4.44
2,41865,2005.0,"Twilight (Twilight, #1)",3.57
3,2657,1960.0,To Kill a Mockingbird,4.25
4,4671,1925.0,The Great Gatsby,3.89


In [493]:
ratings = pd.read_csv("/kaggle/input/goodbooks-10k/ratings.csv")

In [494]:
df = pd.merge(books, ratings, on="book_id", how="inner")

In [495]:
df.head().T

,0,1,2,3,4
book_id,3,3,3,3,3
original_publication_year,1997.0,1997.0,1997.0,1997.0,1997.0
title,Harry Potter and the Sorcerer's Stone (Harry P...,Harry Potter and the Sorcerer's Stone (Harry P...,Harry Potter and the Sorcerer's Stone (Harry P...,Harry Potter and the Sorcerer's Stone (Harry P...,Harry Potter and the Sorcerer's Stone (Harry P...
average_rating,4.44,4.44,4.44,4.44,4.44
user_id,314,588,2077,2487,2900
rating,3,1,2,3,3


In [496]:
df.shape

(79701, 6)

# <p style="background-color:lightgray; font-family:verdana; font-size:250%; text-align:center; border-radius: 15px 20px;">Collaborative Filtering</p>

<div style="border-radius:10px; border:#D0C2F0 solid; padding: 15px; background-color: #FFF0F4; font-size:100%; text-align:left">

<h3 align="left"><font color='#5E5273'>🔍 Item-Based Recommendation</font></h3>


<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">

<h3 align="left"><font color='#DEB887'>💡 Notes:</font></h3>

Item-based recommendation is a type of recommendation system that suggests items to users based on the similarities between items themselves, rather than on the preferences or behaviors of other users. It is a commonly used technique in the field of recommender systems, which are used in various online platforms to suggest products, services, or content to users based on their preferences or behaviors.

In [497]:
#how many rating on per book id? 
df["book_id"].value_counts()

book_id
3       100
415     100
6538    100
5128    100
3061    100
       ... 
6819     60
9762     60
9418     59
8964     59
9534     57
Name: count, Length: 812, dtype: int64

In [498]:
#We are printing the newly reduced dataset with users in rows and titles in columns using a pivot table. 
#We have provided the ratings as the intersections.
user_book_df = df.groupby(["user_id","title"])["rating"].mean().unstack().notnull()
user_book_df

title,'Salem's Lot,"'Tis (Frank McCourt, #2)",1421: The Year China Discovered America,1776,1984,A Bend in the River,A Bend in the Road,A Brief History of Time,A Briefer History of Time,A Case of Need,...,"Women in Love (Brangwen Family, #2)",World War Z: An Oral History of the Zombie War,"World Without End (The Kingsbridge Series, #2)",Wuthering Heights,"Xenocide (Ender's Saga, #3)",Year of Wonders,You Shall Know Our Velocity!,Zen and the Art of Motorcycle Maintenance: An Inquiry Into Values,Zodiac,number9dream
user_id,,,,,,,,,,,,,,,,,,,,,
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53419,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
53420,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
53422,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [499]:
#we took random book name from our dataset
sample = pd.Series(user_book_df.columns).sample(1, random_state=42).values[0]
sample

'Heidi'

In [500]:
#We take the other bookworms votes if they give rate to Heidi.
sample_name = user_book_df[sample]

In [501]:
#corelating heidi book with all otherbooks and sorting values from high to low, first 5
user_book_df.corrwith(sample_name).sort_values(ascending=False).head(5)

title
Heidi                                                           1.000000
Harry Potter Collection (Harry Potter, #1-6)                    0.528368
Harry Potter and the Order of the Phoenix (Harry Potter, #5)    0.518334
Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)     0.488230
Harry Potter and the Half-Blood Prince (Harry Potter, #6)       0.488230
dtype: float64

<center><span style="color:#f7c297;font-family:cursive;font-size:100%"> Our System Recommended Harry Potter </span></center>
<center><img src="https://i.imgur.com/G6WlVxu.jpg" width="800" height="800"></center>


<div style="border-radius:10px; border:#D0C2F0 solid; padding: 15px; background-color: #FFF0F4; font-size:100%; text-align:left">

<h3 align="left"><font color='#5E5273'>🔍 User-Based  Recommendation</font></h3>


<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">

<h3 align="left"><font color='#DEB887'>💡 Notes:</font></h3>
    User-based recommendation is a type of recommendation system that suggests items to users based on the preferences or behaviors of similar users. It relies on the assumption that users with similar tastes or preferences in the past will have similar tastes in the future. User-based recommendation systems are widely used in various online platforms to personalize the user experience and improve user engagement.

In [502]:
#creating new dataframe with user's ratings 
user_df = df.groupby(["user_id","title"])["rating"].mean().unstack()
user_df

title,'Salem's Lot,"'Tis (Frank McCourt, #2)",1421: The Year China Discovered America,1776,1984,A Bend in the River,A Bend in the Road,A Brief History of Time,A Briefer History of Time,A Case of Need,...,"Women in Love (Brangwen Family, #2)",World War Z: An Oral History of the Zombie War,"World Without End (The Kingsbridge Series, #2)",Wuthering Heights,"Xenocide (Ender's Saga, #3)",Year of Wonders,You Shall Know Our Velocity!,Zen and the Art of Motorcycle Maintenance: An Inquiry Into Values,Zodiac,number9dream
user_id,,,,,,,,,,,,,,,,,,,,,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53422,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [503]:
# we took random sample
random_user = user_df.sample(1,random_state=689).index[0]

In [504]:
# sample user df created  with all books
random_user_df = user_df[user_df.index == random_user]
random_user_df

title,'Salem's Lot,"'Tis (Frank McCourt, #2)",1421: The Year China Discovered America,1776,1984,A Bend in the River,A Bend in the Road,A Brief History of Time,A Briefer History of Time,A Case of Need,...,"Women in Love (Brangwen Family, #2)",World War Z: An Oral History of the Zombie War,"World Without End (The Kingsbridge Series, #2)",Wuthering Heights,"Xenocide (Ender's Saga, #3)",Year of Wonders,You Shall Know Our Velocity!,Zen and the Art of Motorcycle Maintenance: An Inquiry Into Values,Zodiac,number9dream
user_id,,,,,,,,,,,,,,,,,,,,,
20467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [505]:
#took only books that our sample read
book_read = random_user_df.dropna(axis=1).columns.tolist()
book_read

['Burmese Days',
 'Daniel Deronda',
 'Freakonomics: A Rogue Economist Explores the Hidden Side of Everything (Freakonomics, #1)',
 'Harry Potter and the Half-Blood Prince (Harry Potter, #6)',
 'Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)',
 'Heidi',
 'I am Charlotte Simmons',
 'Me Talk Pretty One Day',
 'Quicksilver (The Baroque Cycle, #1)',
 'The 158-Pound Marriage',
 'The Broken Wings',
 'The Corrections',
 'The Elegant Universe: Superstrings, Hidden Dimensions, and the Quest for the Ultimate Theory',
 'The Fellowship of the Ring (The Lord of the Rings, #1)',
 "The Hitchhiker's Guide to the Galaxy (Hitchhiker's Guide to the Galaxy, #1)",
 'The Known World',
 'The Long Dark Tea-Time of the Soul (Dirk Gently, #2)',
 'The Lord of the Rings (The Lord of the Rings, #1-3)',
 'The Lord of the Rings: The Art of The Fellowship of the Ring',
 'The Phantom Tollbooth',
 'Tropic of Cancer']

In [506]:
#we check the books of our sample that read is readen by other users
book_read_df = user_df[book_read]
book_read_df

title,Burmese Days,Daniel Deronda,"Freakonomics: A Rogue Economist Explores the Hidden Side of Everything (Freakonomics, #1)","Harry Potter and the Half-Blood Prince (Harry Potter, #6)","Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)",Heidi,I am Charlotte Simmons,Me Talk Pretty One Day,"Quicksilver (The Baroque Cycle, #1)",The 158-Pound Marriage,...,The Corrections,"The Elegant Universe: Superstrings, Hidden Dimensions, and the Quest for the Ultimate Theory","The Fellowship of the Ring (The Lord of the Rings, #1)","The Hitchhiker's Guide to the Galaxy (Hitchhiker's Guide to the Galaxy, #1)",The Known World,"The Long Dark Tea-Time of the Soul (Dirk Gently, #2)","The Lord of the Rings (The Lord of the Rings, #1-3)",The Lord of the Rings: The Art of The Fellowship of the Ring,The Phantom Tollbooth,Tropic of Cancer
user_id,,,,,,,,,,,,,,,,,,,,,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53422,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [507]:
#how many same books readen by other readers?
user_book_count = book_read_df.T.notnull().sum()
user_book_count

user_id
2        0
3        0
4        0
7        0
9        0
        ..
53419    0
53420    0
53422    0
53423    0
53424    0
Length: 28906, dtype: int64

In [508]:
#Reader IDs of people who read books with more than 30% similarity.
users_same_books = user_book_count[user_book_count > (book_read_df.shape[1] * 30 ) / 100].index
users_same_books

Index([  588,  1952,  5461,  6342,  9246, 10111, 10727, 10944, 11692, 11927,
       11945, 12381, 13544, 13794, 17984, 18031, 18361, 20467, 22602, 23576,
       23612, 24326, 26244, 26661, 28767, 29703, 32592, 32748, 32923, 33065,
       33716, 33872, 38080, 42404, 44397, 45554, 47800, 48559, 48687, 51166],
      dtype='int64', name='user_id')

In [509]:
#we merged our sample's data and whole data together.
filted_df = book_read_df[book_read_df.index.isin(users_same_books)]
filted_df.head()

title,Burmese Days,Daniel Deronda,"Freakonomics: A Rogue Economist Explores the Hidden Side of Everything (Freakonomics, #1)","Harry Potter and the Half-Blood Prince (Harry Potter, #6)","Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)",Heidi,I am Charlotte Simmons,Me Talk Pretty One Day,"Quicksilver (The Baroque Cycle, #1)",The 158-Pound Marriage,...,The Corrections,"The Elegant Universe: Superstrings, Hidden Dimensions, and the Quest for the Ultimate Theory","The Fellowship of the Ring (The Lord of the Rings, #1)","The Hitchhiker's Guide to the Galaxy (Hitchhiker's Guide to the Galaxy, #1)",The Known World,"The Long Dark Tea-Time of the Soul (Dirk Gently, #2)","The Lord of the Rings (The Lord of the Rings, #1-3)",The Lord of the Rings: The Art of The Fellowship of the Ring,The Phantom Tollbooth,Tropic of Cancer
user_id,,,,,,,,,,,,,,,,,,,,,
588,NaN,NaN,NaN,5.0,NaN,NaN,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,4.0,4.0,NaN,3.0,NaN,4.0,4.0
1952,NaN,4.0,NaN,NaN,5.0,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5.0,4.0,NaN,4.0,5.0,4.0,NaN
5461,NaN,4.0,4.0,3.0,5.0,5.0,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,4.0,4.0,NaN,NaN,NaN
6342,NaN,4.0,4.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5.0,5.0,NaN,4.0,4.0,NaN,NaN
9246,NaN,NaN,NaN,1.0,3.0,NaN,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,5.0,5.0,3.0,4.0,2.0,5.0,NaN


In [510]:
#made a correlation df 
corr_df = filted_df.T.corr().unstack().drop_duplicates()
corr_df

user_id  user_id
588      588        1.000000
         1952       0.333333
         5461      -0.774597
         6342       1.000000
         9246      -0.161165
                      ...   
44397    51166      0.482382
45554    48559      0.157243
         51166     -0.904534
47800    48559      0.321288
48559    51166     -0.166667
Length: 582, dtype: float64

In [511]:
# we get thoose people who is high correlated with our bookworm
top_readers = pd.DataFrame(corr_df[random_user][corr_df[random_user] > 0.65], columns=["corr"])
top_readers

,corr
user_id,
23612,0.871044
26661,0.782624
33716,0.744157


In [512]:
# we get ratings of top readers 
top_readers_ratings = pd.merge(top_readers, df[["user_id", "book_id", "rating"]], how='inner', on="user_id")
top_readers_ratings

,user_id,corr,book_id,rating
0,23612,0.871044,3,3
1,23612,0.871044,34,1
2,23612,0.871044,2,5
3,23612,0.871044,968,3
4,23612,0.871044,1,4
...,...,...,...,...
72,33716,0.744157,36,3
73,33716,0.744157,647,3
74,33716,0.744157,304,5
75,33716,0.744157,6670,3


In [513]:
# weighted the ratings of top readers
top_readers_ratings['weighted_rating'] = top_readers_ratings['corr'] * top_readers_ratings['rating']
top_readers_ratings

,user_id,corr,book_id,rating,weighted_rating
0,23612,0.871044,3,3,2.613133
1,23612,0.871044,34,1,0.871044
2,23612,0.871044,2,5,4.355222
3,23612,0.871044,968,3,2.613133
4,23612,0.871044,1,4,3.484178
...,...,...,...,...,...
72,33716,0.744157,36,3,2.232472
73,33716,0.744157,647,3,2.232472
74,33716,0.744157,304,5,3.720787
75,33716,0.744157,6670,3,2.232472


In [514]:
# Made a recommendation df with book id and weighted ratings
recommendation_df = top_readers_ratings.pivot_table(values="weighted_rating", index="book_id", aggfunc="mean")
recommendation_df.head()

,weighted_rating
book_id,
1,3.445153
2,4.038004
3,2.422803
5,3.444874
8,3.444874


In [515]:
# Recommending books by higher than 3.5 rating, sorted values high to low
books_recommend = recommendation_df[recommendation_df["weighted_rating"] > 3.5].sort_values(by="weighted_rating", ascending=False).head()
books_recommend

,weighted_rating
book_id,
21,4.355222
24,4.355222
25,4.355222
27,4.355222
1274,4.355222


In [516]:
# Thats the name of the books that we can recommend higher than 3.5 weighted rating
books[books["book_id"].isin(books_recommend.index)]

,book_id,original_publication_year,title,average_rating
373,21,2003.0,A Short History of Nearly Everything,4.19
572,1274,1998.0,"Men Are from Mars, Women Are from Venus",3.52
1459,24,2000.0,In a Sunburned Country,4.05
1975,25,1998.0,I'm a Stranger Here Myself: Notes on Returning...,3.89
2278,27,1991.0,Neither Here nor There: Travels in Europe,3.88


<center><span style="color:#f7c297;font-family:cursive;font-size:100%"> first 5 Books that recommended </span></center>
<center><img src="https://i.imgur.com/GoYs5gY.png" width="800" height="800"></center>

<div style="border-radius:10px; border:#D0C2F0 solid; padding: 15px; background-color: #FFF0F4; font-size:100%; text-align:left">

<h3 align="left"><font color='#5E5273'>🔍 Model Based Recommendation</font></h3>


<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">

<h3 align="left"><font color='#DEB887'>💡 Notes:</font></h3>
    
Model-based recommendation is a type of recommendation system that relies on building a predictive model based on the user-item interactions or other relevant features present in the dataset. Unlike memory-based collaborative filtering techniques that directly use the user-item interactions, model-based approaches create a model to predict user preferences and provide recommendations based on this model.

In [517]:
#we took sample
user_ids = df["user_id"].sample(1,random_state=42).values.tolist()[0]
user_ids

45029

In [518]:
#books that our sample read
sample_df = df[df["user_id"]==user_ids]
sample_df

,book_id,original_publication_year,title,average_rating,user_id,rating
5311,2165,1952.0,The Old Man and the Sea,3.73,45029,4
7552,5359,1993.0,The Client,3.97,45029,4
19233,2373,1997.0,"The Bone Collector (Lincoln Rhyme, #1)",4.18,45029,4
61309,4630,1937.0,To Have and Have Not,3.57,45029,4
62878,5548,1988.0,What Do You Care What Other People Think?,4.28,45029,4


In [519]:
#we gave a scale that beetwen 1 and 5 for ratings
reader = Reader(rating_scale=(1, 5))

In [520]:
#making new dataset for Surprise library
data = Dataset.load_from_df(df[['user_id','book_id','rating']], reader)

In [521]:
#train test split
trainset, testset = train_test_split(data,random_state=42, test_size=.25)

In [522]:
#build model and fitting it to our train set
svd_model = SVD(random_state=42)
svd_model.fit(trainset)

In [523]:
#predictions of our model made on test set
predictions = svd_model.test(testset)

In [524]:
#books that our sample didnt read
didnt_read = df["book_id"][~(df["user_id"] == user_ids)].drop_duplicates().values.tolist()

In [525]:
#function that recommends user which didnt read but gave high score to the book from our machine learning model
def suggest(df,user_id,sug):
    didnt_read = df["book_id"][~(df["user_id"]==user_id)].drop_duplicates().values.tolist()
    temp_dict={}
    for i in didnt_read:
        temp_dict[i] = svd_model.predict(uid=user_id, iid=i)[3]
    suggestions = pd.DataFrame(temp_dict.items(),columns=["book_id",'possible_rate']).sort_values(by="possible_rate", ascending=False).head(sug)
    merged = pd.merge(suggestions,books[["book_id","title"]], how="inner", on="book_id")
    return merged

In [526]:
#for our sample, our machine learning model suggested 5 different book which our model can give around 4.7 rating.
suggest(df, user_ids, 5)

,book_id,possible_rate,title
0,9566,4.823831,Still Life with Woodpecker
1,9531,4.760682,Peter and the Shadow Thieves (Peter and the St...
2,3885,4.718368,The Taste of Home Cookbook
3,4708,4.709676,The Beautiful and Damned
4,6423,4.662150,"High Five (Stephanie Plum, #5)"


<center><span style="color:#f7c297;font-family:cursive;font-size:100%"> Books that our Model Recommends </span></center>
<center><img src="https://i.imgur.com/ftZF73j.png" width="800" height="800"></center>